In [1]:
import numpy as np

r = np.random.beta(0.1, 1)
print r

1.99104426988e-11


In [110]:
from xml.etree import cElementTree as ET
import sys
from HTMLParser import HTMLParser

# credit: http://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

posts = open('Posts.xml', 'r').read()

posts[1:100]

def remove_tags(text):
    return ''.join(ET.fromstring(text).itertext())

root = ET.fromstring(posts)
documents = []
for child in root.findall('row')[0:100]:
    text = None
    child_text = child.get('Body').encode('utf-8').strip()
    text = strip_tags(child_text)
    documents.append(text)

print documents[0:10]

['I was offered a beer the other day that was reportedly made with citra hops. What are citra hops? Why should I care that my beer is made with them?', 'As far as we know, when did humans first brew beer, and where? Around when would you have been able to get your hands on something resembling a modern lager?', "How is low/no alcohol beer made? I'm assuming that the beer is made normally and the alcohol is then removed, is it any more than just boiling it off? I've noticed that no/low alcohol beers' taste improved hugely a few years ago, is this due to a new technique?", 'Citra is a registered trademark since 2007. Citra Brand hops have fairly high alpha acids and total oil contents with a low percentage of cohumulone content and  imparts interesting citrus and tropical fruit characters to beer.\n\nFor more information, you can read the Wikipedia article on the Citra brand.', "In general, what's the best way to work out the temperature at which to serve a particular beer? Room temperat

In [111]:
execfile('corpus.py')
execfile('document.py')

corpus = Corpus(documents, '../Week1HW/stopwords.txt', 3)

print corpus.docs[0].tokens
# FIXME: Not sure why this is not 100
print corpus.N

[u'wa' u'offer' u'beer' u'the' u'day' u'wa' u'reportedli' u'citra' u'hop'
 u'are' u'citra' u'hop' u'whi' u'care' u'beer']
100


## Algorithm

#### Initialization

1. Randomly assign all words to a topic
2. Initialize theta (D x K, document-specific topic probabilities): each theta_d is a k-dimensional sample from Dir(alpha + number of words in topic k)
3. Initialize beta (K x V, topic-specific term probabilities): each beta_k is a V-dimensional sample from Dir(eta + number of times term v appeared in topic k)

#### Repeat

1. for every word wi in N, it's current allocation is zi
    1.1. decrement -1 zi document-topic count and topic-term count
    (Question) Do we update theta, beta for these reduced counts?
    1.2. Use theta and beta to calculate probability over topic for word (slide 14 lecture6)
    1.3. draw from multinomial with these probabilites - this is the new allocation zi for word wi
    1.4. increment +1 zi document-topic count and topic-term count
2. update theta and beta


In [112]:
# Initializations
#
D = len(corpus.docs)
K = 3 # K - numer of topics
V = len(corpus.token_set)

# Randomly assign each word in each document to a topic
# each array is of "arbitrary" length - length of document's respective word list
word_topic_assignments = [[]]*corpus.N
for doci, doc in enumerate(corpus.docs):
    word_topic_assignments[doci] = []
    for wordi, word in enumerate(doc.tokens):
        word_topic_assignments[doci].append(np.random.randint(0,K))

#print word_topic_assignments[0]

# Initialize document counts of words for each topic, D x K
document_topic_word_distribution = np.zeros((D, K))
for doci, doc in enumerate(corpus.docs):
    # sum the number of words in topic k
    for k in range(0,K):
        document_topic_word_distribution.itemset((doci, k), word_topic_assignments[doci].count(k))

#print document_topic_word_distribution[0,:]

# Initialize term topic counts, e.g. number of times term V was allocated to topic K, K x V
# For every word in every document, determine what it's term index is
topic_term_distribution = np.zeros((K, V))
termlist = list(corpus.token_set)
for doci, doc in enumerate(corpus.docs):
    for wordi, word in enumerate(doc.tokens):
        termidx = termlist.index(word)
        topic_alloc = word_topic_assignments[doci][wordi]
        current_count = topic_term_distribution.item((topic_alloc,termidx))
        topic_term_distribution.itemset((topic_alloc,termidx), current_count + 1)
        
#print topic_term_distribution[0,:][0:10]

alpha = 50/K
eta = 200/V

# Initialize theta - document-specific topic probabilities
theta = np.zeros((D, K))
# draw theta - Dirichlet with paramters (alpha + n_{d,k}) - num of words in doc with topic alloc k
for doci in range(D):
    theta_params = alpha + document_topic_word_distribution[doci,:]
    # theta for this document
    theta_d = np.random.dirichlet(tuple(theta_params), 1)
    theta[doci,:] = theta_d

# Initialize beta - x
beta = np.zeros((K, V))
# draw beta - Dirichlet with paramters (eta + m_{k,v}) - num of times term appeared in topic k
for k in range(K):
    beta_params = eta + topic_term_distribution[k,:]
    beta_k = np.random.dirichlet(tuple(beta_params), 1)
    beta[k,:] = beta_k
    
terms_to_sanity_check = ['skunk', 'can', 'wine', 'saison']
for term in terms_to_sanity_check:
    termidx = termlist.index(term)    
    print 'Term: ' + term
    print 'Original topic distribution: ' + str(topic_term_distribution[:,termidx])

iters = 100
for i in range(iters):
    if i%25 == 0: print 'Iteration ' + str(i)
    # 1 iteration
    for doci, doc in enumerate(corpus.docs):
        theta_d = theta[doci,:]
        if i == 0 and doci == 0: print theta_d
        for wordi, word in enumerate(doc.tokens):
            word_topic_alloc = word_topic_assignments[doci][wordi]
            termidx = termlist.index(word)
            beta_v = beta[:,termidx]
            if i == 0 and doci == 0: print beta_v            
            # decrement counts
            current_doc_count = document_topic_word_distribution.item((doci, word_topic_alloc))
            document_topic_word_distribution.itemset((doci, word_topic_alloc), current_doc_count - 1)
            current_topic_count = topic_term_distribution.item((word_topic_alloc, termidx))
            topic_term_distribution.itemset((word_topic_alloc, termidx), current_topic_count - 1)
            probs = theta_d*beta_v/np.sum(theta_d*beta_v)
            new_topic_alloc = np.random.multinomial(1, tuple(probs))
            new_topic_alloc = list(new_topic_alloc).index(1)                
            word_topic_assignments[doci][wordi] = new_topic_alloc
            # increment counts
            current_doc_count = document_topic_word_distribution.item((doci, new_topic_alloc))
            document_topic_word_distribution.itemset((doci, new_topic_alloc), current_doc_count + 1)
            current_topic_count = topic_term_distribution.item((new_topic_alloc, termidx))
            topic_term_distribution.itemset((new_topic_alloc, termidx), current_topic_count + 1) 
    # update theta
    for doci in range(D):
        theta_params = alpha + document_topic_word_distribution[doci,:]
        # theta for this document
        theta_d = np.random.dirichlet(tuple(theta_params), 1)
        theta[doci,:] = theta_d
    for k in range(K):
        beta_params = eta + topic_term_distribution[k,:]
        beta_k = np.random.dirichlet(tuple(beta_params), 1)
        beta[k,:] = beta_k    
        
for termi, term in enumerate(terms_to_sanity_check):
    termidx = termlist.index(term)
    print 'Term: ' + term
    print 'New topic distribution: ' + str(topic_term_distribution[:,termidx])


Term: skunk
Original topic distribution: [ 6.  2.  2.]
Term: can
Original topic distribution: [ 26.  15.  19.]
Term: wine
Original topic distribution: [ 3.  8.  3.]
Term: saison
Original topic distribution: [ 0.  3.  1.]
Iteration 0
[ 0.3715127   0.24354962  0.38493769]
[ 0.00521869  0.00727849  0.00084276]
[  8.96499578e-05   6.08596577e-05   2.61853725e-05]
[ 0.0454536   0.04743505  0.04645351]
[ 0.06644518  0.0594474   0.07233897]
[ 0.00030227  0.00131574  0.00094881]
[ 0.00521869  0.00727849  0.00084276]
[ 0.          0.          0.00090331]
[ 0.00345078  0.00022846  0.00143478]
[ 0.00123987  0.00335368  0.00385033]
[ 0.01296966  0.01946525  0.01812061]
[ 0.00345078  0.00022846  0.00143478]
[ 0.00123987  0.00335368  0.00385033]
[ 0.00446502  0.00351178  0.00181186]
[  3.93915273e-04   9.62839212e-05   0.00000000e+00]
[ 0.0454536   0.04743505  0.04645351]
Iteration 25
Iteration 50
Iteration 75
Term: skunk
New topic distribution: [ 10.   0.   0.]
Term: can
New topic distribution: [ 2

In [113]:
# Print the most likely words for every topic
sorted_beta = np.zeros(beta.shape)

for k in range(0,K):
    print ''
    print 'Most likely terms for topic ' + str(k) + ':'
    sorted_beta[k,:] = np.argsort(beta[k,:])[::-1]
    for i in range(20):
        idx = int(sorted_beta[k,:][i])
        print termlist[idx]


Most likely terms for topic 0:
for
the
beer
alcohol
use
can
ale
ani
stout
lager
how
gener
brew
glass
out
much
caus
plastic
mani
higher

Most likely terms for topic 1:
and
are
you
the
bottl
thi
drink
yeast
will
ipa
all
abv
becaus
wine
keep
brewer
imperi
carbon
depend
gluten

Most likely terms for topic 2:
the
beer
but
ferment
not
can
differ
temperatur
tast
wa
get
flavor
process
ha
exampl
good
may
age
don
doe
